In [112]:
# GenIce用に作った、cif読み込みルーチン pip install cif2ice
from cif2ice import read_cif

atoms, cell = read_cif.read_and_process("MethaneA.cif")
atoms, cell

([('H', 0.025000000000000355, 0.6750000000000007, 0.11800000000000033),
  ('H', 0.6750000000000007, 0.11800000000000033, 0.025000000000000355),
  ('H', 0.08999999999999986, 0.6189999999999998, 0.2759999999999998),
  ('H', 0.6189999999999998, 0.2759999999999998, 0.08999999999999986),
  ('H', 0.1869999999999994, 0.7289999999999992, 0.17099999999999937),
  ('H', 0.7289999999999992, 0.17099999999999937, 0.1869999999999994),
  ('H', 0.0389999999999997, 0.7899999999999991, 0.2530000000000001),
  ('H', 0.7899999999999991, 0.2530000000000001, 0.0389999999999997),
  ('H', 0.006000000000000227, 0.3509999999999991, 0.9890000000000008),
  ('H', 0.3509999999999991, 0.9890000000000008, 0.006000000000000227),
  ('H', 0.07900000000000063, 0.3990000000000009, 0.8320000000000007),
  ('H', 0.3990000000000009, 0.8320000000000007, 0.07900000000000063),
  ('H', 0.031000000000000583, 0.5240000000000009, 0.9510000000000005),
  ('H', 0.5240000000000009, 0.9510000000000005, 0.031000000000000583),
  ('H', 0.1720

原子の順番がソートされてしまっているので、炭素と水素をくっつける作業が必要。

これが自動化できない原因か。CIFはX線で得られた構造の情報で、分子とか結合とか知ったことじゃないので、順番に意味がない。でもGromacsではTopologyファイルで定義される順番が重要。

鏡像になっても動いてくれるはず。

In [113]:
carbons = np.array([atom[1:4] for atom in atoms if atom[0] == "C"])
carbons

array([[0.085  , 0.703  , 0.205  ],
       [0.703  , 0.205  , 0.085  ],
       [0.072  , 0.423  , 0.939  ],
       [0.423  , 0.939  , 0.072  ],
       [0.189  , 0.435  , 0.519  ],
       [0.435  , 0.519  , 0.189  ],
       [0.626  , 0.953  , 0.72   ],
       [0.953  , 0.72   , 0.626  ],
       [0.318  , 0.839  , 0.4454 ],
       [0.839  , 0.4454 , 0.318  ],
       [0.291  , 0.701  , 0.834  ],
       [0.701  , 0.834  , 0.291  ],
       [0.205  , 0.085  , 0.703  ],
       [0.939  , 0.072  , 0.423  ],
       [0.519  , 0.189  , 0.435  ],
       [0.72   , 0.626  , 0.953  ],
       [0.4454 , 0.318  , 0.839  ],
       [0.834  , 0.291  , 0.701  ],
       [0.187  , 0.187  , 0.187  ],
       [0.954  , 0.954  , 0.954  ],
       [0.57578, 0.57577, 0.57578]])

In [114]:
hydrogens = np.array([atom[1:4] for atom in atoms if atom[0] == "H"])

neighbors = []
for i in range(len(carbons)):
    CH = hydrogens - carbons[i]
    CH -= np.floor(CH+0.5)
    CH = CH @ cellmat
    CHlen = np.sum(CH*CH, axis=1)
    neighbors.append(np.argwhere(CHlen < 1.0).reshape(-1))

neighbors

[array([0, 2, 4, 6]),
 array([1, 3, 5, 7]),
 array([ 8, 10, 12, 14]),
 array([ 9, 11, 13, 15]),
 array([16, 18, 20, 22]),
 array([17, 19, 21, 23]),
 array([24, 26, 28, 30]),
 array([25, 27, 29, 31]),
 array([32, 34, 36, 38]),
 array([33, 35, 37, 39]),
 array([40, 42, 44, 46]),
 array([41, 43, 45, 47]),
 array([54, 55, 56, 57]),
 array([58, 59, 60, 61]),
 array([62, 63, 64, 65]),
 array([66, 67, 68, 69]),
 array([70, 71, 72, 73]),
 array([74, 75, 76, 77]),
 array([48, 49, 78, 79]),
 array([50, 51, 80, 81]),
 array([52, 53, 82, 83])]

In [115]:
import gromacs
from genice2.cell import cellvectors

cellmat = cellvectors(*cell)
cellmat /= 10  # angstrom to nm

# to absolute coord
carbons = carbons @ cellmat
hydrogens = hydrogens @ cellmat

atom = []
positions = []
for ci, carbon in enumerate(carbons):
    atom.append("C")
    positions.append(carbon)
    for hi in neighbors[ci]:
        atom.append("H")
        positions.append(hydrogens[hi])
        

residue = ["MET" for _ in atoms]
resi_id = [(i//5)+1 for i in range(len(atom))]
atom_id = [i+1 for i in range(len(atom))]

frame = {
    "residue": residue,
    "atom": atom,
    "position": positions,
    "resi_id": resi_id,
    "atom_id": atom_id,
    "cell": cellmat,
}

with open("MethaneA.gro", "w") as f:
    gromacs.write_gro(frame, f)

In [13]:
# groファイルを読みこんでセルを4x4x4倍にするだけ

import gromacs
import numpy as np

with open("MethaneA.gro") as f:
    for frame in gromacs.read_gro(f):
        break

repeat = (4, 4, 4)
cellmat = frame["cell"]
celli = np.linalg.inv(cellmat)
unitcell = frame["position"] @ celli / np.array(repeat)
print(unitcell)

positions = [
    unitcell + np.array([x / repeat[0], y / repeat[1], z/repeat[2]])
    for x in range(repeat[0])
    for y in range(repeat[1])
    for z in range(repeat[2])
]

numAtoms = len(unitcell) * repeat[0] * repeat[1] * repeat[2]


cellmat *= np.array(repeat)
frame = {
    "position" : np.vstack(positions) @ cellmat,
    "resi_id": [(i//5)+1 for i in range(numAtoms)],
    "atom_id": [i+1 for i in range(numAtoms)],
    "atom": frame["atom"] * repeat[0] * repeat[1] * repeat[2],
    "residue": frame["residue"] * repeat[0] * repeat[1] * repeat[2],
    "cell": cellmat
}

with open("MethaneA444.gro", "w") as f:
    gromacs.write_gro(frame, f)




[[0.02123002 0.17584835 0.0511262 ]
 [0.00620428 0.16869101 0.02938287]
 [0.02243223 0.15483224 0.06904975]
 [0.04680849 0.18238606 0.04260517]
 [0.00977141 0.19748665 0.06317318]
 [0.17565221 0.05123509 0.02115567]
 [0.16862643 0.02962221 0.0061704 ]
 [0.15462283 0.06914526 0.02262481]
 [0.18226041 0.04278704 0.04671877]
 [0.19738791 0.06338075 0.00969635]
 [0.01788828 0.10580011 0.23476916]
 [0.00148475 0.08777047 0.24710997]
 [0.01964247 0.09986126 0.20803075]
 [0.00765293 0.13104302 0.23770745]
 [0.04307697 0.10394316 0.24564083]
 [0.1056233  0.23460532 0.01792355]
 [0.0877332  0.24738183 0.00146914]
 [0.09966888 0.20814647 0.01968653]
 [0.130954   0.23763247 0.00763955]
 [0.10413202 0.24584815 0.042899  ]
 [0.04727703 0.10876431 0.1298723 ]
 [0.05880041 0.11001733 0.15484775]
 [0.02507602 0.09288798 0.13104762]
 [0.04037015 0.13410381 0.1216451 ]
 [0.06544916 0.09804811 0.11194875]
 [0.10863235 0.12975201 0.04730643]
 [0.11007812 0.15462738 0.05876575]
 [0.09294973 0.13112047 0.02